In [142]:
import os
os.chdir('/home/jovyan/work')
print(os.getcwd())
import init
import reading
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u

def getCandidates():
    df = pd.DataFrame.from_csv(init.basedir+'upsilon_output.txt')
    df.sort_values(by='probability', ascending=False)
    df=df[df['label'] != 'NonVar']
    df=df[df["probability"] > 0.50]
    df=df[df["flag"] != 1]

    positions=reading.read_world_positions(init.worldposdir)
    result = []
    for index, row in df.iterrows():
        #print(index, row)
        result.append([index, row['label'], row['probability'], row['flag'], SkyCoord(positions[int(index)][0], positions[int(index)][1], unit='deg')])
    return result

# returns [name, skycoord, type]
def getVSX():
    result = []
    df = pd.DataFrame.from_csv(init.basedir+'SearchResults.csv')
    #print(df.head())
    for index, row in df.iterrows():
        skycoord = SkyCoord(row['Coords'], unit=(u.hourangle, u.deg))
        result.append([index, skycoord, row['Type']])
    return result

def findMatch(vsx, candidates):
    for candidate in candidates:
        best_sep_deg = 360
        best_sep_string = ""
        best_var = None
        for variable in vsx:
            sep = candidate[4].separation(variable[1])
            if(sep.degree < best_sep_deg):
                best_sep_deg = sep.degree
                best_sep_string = sep.to_string()
                best_var = variable
        candidate.append([best_var[0],best_var[1],best_var[2],best_sep_deg, best_sep_string])
    return candidates
        
    
matches = findMatch(getVSX(), getCandidates())
print(matches)
variables_star_list = [i[0] for i in matches]
print(variables_star_list)

/home/jovyan/work
[[138, 'RRL_ab', 0.96, 0, <SkyCoord (ICRS): (ra, dec) in deg
    (271.40328856, -43.83223887)>, ['WW CrA', <SkyCoord (ICRS): (ra, dec) in deg
    (271.40329167, -43.83261111)>, 'RRAB/BL', 0.0003722439468105054, '0d00m01.3401s']], [73, 'CEPH_Other', 0.51, 0, <SkyCoord (ICRS): (ra, dec) in deg
    (271.37481064, -44.12829733)>, ['ASASSN-V J180536.53-441521.6', <SkyCoord (ICRS): (ra, dec) in deg
    (271.40220833, -44.256)>, 'RRAB', 0.1292047590730617, '0d07m45.1371s']]]
[138, 73]


In [150]:
matches[0][4].ra.hms

hms_tuple(h=18.0, m=5.0, s=36.78925405505993)